In [ ]:
!pip install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 11.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import geopandas as gpd
import matplotlib.pyplot as plt
import pycountry

In [ ]:
# Connect With Drive
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

# %cd to api key
%cd /content/drive/MyDrive/NAACL 2024 /Scientometrics
!ls

Mounted at /content/drive
/content/drive/MyDrive/NAACL 2024 /Scientometrics
'1. Global Health PMIDs Extraction.ipynb'
 Data_extraction.ipynb
 EDA.ipynb
 gender-api-output.csv
 HICs_preprocessed_global_health_papers_2018_2023.csv
'List of economies.csv'
 LMICs_preprocessed_global_health_papers_2018_2023.csv
 papers_2021_2024.csv
 Papers_Info.csv
 Papers_Info_raw.csv
 Preprocessed_papers_full.csv
 Preprocessing.ipynb


In [ ]:
data = pd.read_csv('Papers_Info_raw.csv')
#data = data[data['year'] > 2017]
data = data.dropna(subset=['journal'])
data.head()

,authors,date,doi,id,journal,linkout,source_title,title,year,funder_countries,funders
0,"[{'affiliations': [{'city': 'Ann Arbor', 'city...",2024-06-07,10.2196/58370,pub.1172744332,"{'id': 'jour.1053709', 'title': 'JMIR Medical ...",http://dx.doi.org/10.2196/58370,"{'id': 'jour.1053709', 'title': 'JMIR Medical ...",Authors' Reply: A Use Case for Generative AI i...,2024,NaN,NaN
1,"[{'affiliations': [{'city': 'Boston', 'city_id...",2024-06-11,10.1542/peds.2023-065573,pub.1172744193,"{'id': 'jour.1017874', 'title': 'Pediatrics'}",NaN,"{'id': 'jour.1017874', 'title': 'Pediatrics'}",Performance of ChatGPT and Google Translate fo...,2024,NaN,NaN
2,"[{'affiliations': [{'city': 'Munich', 'city_id...",2024-06-10,10.1101/cshperspect.a041458,pub.1172743938,"{'id': 'jour.1041980', 'title': 'Cold Spring H...",NaN,"{'id': 'jour.1041980', 'title': 'Cold Spring H...",Artificial Intelligence Learns Protein Predict...,2024,NaN,NaN
3,"[{'affiliations': [{'city': 'Shenzhen', 'city_...",2024-06-10,10.1097/js9.0000000000001796,pub.1172743827,"{'id': 'jour.1295032', 'title': 'International...",NaN,"{'id': 'jour.1295032', 'title': 'International...",Regarding the evaluation of large language mod...,2024,NaN,NaN
4,"[{'affiliations': [{'city': 'Houston', 'city_i...",2024-06-10,10.1093/jamia/ocae129,pub.1172675812,"{'id': 'jour.1111851', 'title': 'Journal of th...",NaN,"{'id': 'jour.1111851', 'title': 'Journal of th...",RefAI: a GPT-powered retrieval-augmented gener...,2024,"[{'id': 'US', 'name': 'United States'}]","[{'acronym': 'NIAID', 'city_name': 'Bethesda',..."


### Preprocessing

In [ ]:
# Load the dataframe
import pandas as pd
import ast
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataframe
df = data

# Function to safely parse the JSON-like string
def safe_json_loads(s):
    # Ignore if is NaN
    if pd.isnull(s):
        return np.nan
    try:
        return json.loads(s.replace("'", '"'))
    except ValueError:
        return ast.literal_eval(s)

# Drop rows with missing author information
df_clean = df.dropna(subset=['authors'])

# Convert the authors' column into a list if it's not a list and split it into different rows
df_clean['authors'] = df_clean['authors'].apply(safe_json_loads)
df_clean['funders'] = df_clean['funders'].apply(safe_json_loads)
df_exploded = df_clean.explode('authors')

# Extract author information into separate columns
def extract_author_info(row):
    author_info = row['authors']
    # Base information
    info = {
        'first_name': author_info.get('first_name', ''),
        'last_name': author_info.get('last_name', ''),
        'first_affiliation': np.nan,
        'second_affiliation': np.nan,
        'third_affiliation': np.nan,
        'first_country': np.nan,
        'first_country_code': np.nan,
        'second_country': np.nan,
        'second_country_code': np.nan,
        'third_country': np.nan,
        'third_country_code': np.nan
    }
    affiliations = author_info.get('affiliations', [])
    if affiliations:
        for i, affiliation in enumerate(affiliations[:3], start=1):
            info[f'first_affiliation' if i == 1 else f'second_affiliation' if i == 2 else 'third_affiliation'] = affiliation.get('name', '')
            info[f'first_country' if i == 1 else f'second_country' if i == 2 else 'third_country'] = affiliation.get('country', '')
            info[f'first_country_code' if i == 1 else f'second_country_code' if i == 2 else 'third_country_code'] = affiliation.get('country_code', '')
    return pd.Series(info)


# Extract founder information into separate columns
def extract_funder_info(row):
    # check if the funder information is np.nan
    if isinstance(row['funders'], (int, float)):

        if pd.isna(row['funders']):
            return pd.Series({
                'funder_name': np.nan,
                'funder_country': np.nan,
                'funder_country_code': np.nan
            })

    funder_info = row['funders'][0]
    # Base information
    info = {
        'funder_name': funder_info.get('name', ''),
        'funder_country': funder_info.get('country_name', ''),
        'funder_country_code': funder_info.get('country_code', ''),
    }

    return pd.Series(info)

# Function to safely parse a JSON-like string into a dictionary
def safe_parse_dict(s):
    try:
        # Attempt to parse as JSON
        return json.loads(s.replace("'", '"'))
    except ValueError:
        try:
            # Fallback to using ast.literal_eval as a last resort
            return ast.literal_eval(s)
        except ValueError:
            # Return None or an empty dict if parsing fails
            return {}


# Apply the function to extract info

# After exploding the dataframe to separate authors
df_exploded.reset_index(drop=True, inplace=True)
# Ensure the extracted author information also has a reset index before concatenation
author_columns = df_exploded.apply(extract_author_info, axis=1).reset_index(drop=True)
# Ensure the extracted funder information also has a reset index before concatenation
funder_columns = df_exploded.apply(extract_funder_info, axis=1).reset_index(drop=True)
# Now, concatenate using the reset indices to ensure proper alignment
df_final = pd.concat([df_exploded, author_columns, funder_columns], axis=1)
df_final['journal'] = df_final['journal'].apply(safe_parse_dict)
df_final['journal_title'] = df_final['journal'].apply(lambda x: x.get('title', '') if isinstance(x, dict) else '')

# Determine the position of first and last authors
df_final['author_position'] = df_final.groupby('doi').cumcount() + 1
group_sizes = df_final.groupby('doi').size()
df_final['last_author_position'] = df_final['doi'].map(group_sizes)

# Create columns for first and last authors
df_final['is_first_author'] = (df_final['author_position'] == 1)
df_final['is_last_author'] = (df_final['author_position'] == df_final['last_author_position'])

df_final.head()

,authors,date,doi,id,journal,linkout,source_title,title,year,funder_countries,...,third_country,third_country_code,funder_name,funder_country,funder_country_code,journal_title,author_position,last_author_position,is_first_author,is_last_author
0,"{'affiliations': [{'city': 'Ann Arbor', 'city_...",2024-06-07,10.2196/58370,pub.1172744332,"{'id': 'jour.1053709', 'title': 'JMIR Medical ...",http://dx.doi.org/10.2196/58370,"{'id': 'jour.1053709', 'title': 'JMIR Medical ...",Authors' Reply: A Use Case for Generative AI i...,2024,NaN,...,NaN,NaN,NaN,NaN,NaN,JMIR Medical Education,1.0,2.0,True,False
1,"{'affiliations': [{'city': 'Evanston', 'city_i...",2024-06-07,10.2196/58370,pub.1172744332,"{'id': 'jour.1053709', 'title': 'JMIR Medical ...",http://dx.doi.org/10.2196/58370,"{'id': 'jour.1053709', 'title': 'JMIR Medical ...",Authors' Reply: A Use Case for Generative AI i...,2024,NaN,...,NaN,NaN,NaN,NaN,NaN,JMIR Medical Education,2.0,2.0,False,True
2,"{'affiliations': [{'city': 'Boston', 'city_id'...",2024-06-11,10.1542/peds.2023-065573,pub.1172744193,"{'id': 'jour.1017874', 'title': 'Pediatrics'}",NaN,"{'id': 'jour.1017874', 'title': 'Pediatrics'}",Performance of ChatGPT and Google Translate fo...,2024,NaN,...,United States,US,NaN,NaN,NaN,Pediatrics,1.0,9.0,True,False
3,"{'affiliations': [{'city': 'Boston', 'city_id'...",2024-06-11,10.1542/peds.2023-065573,pub.1172744193,"{'id': 'jour.1017874', 'title': 'Pediatrics'}",NaN,"{'id': 'jour.1017874', 'title': 'Pediatrics'}",Performance of ChatGPT and Google Translate fo...,2024,NaN,...,United States,US,NaN,NaN,NaN,Pediatrics,2.0,9.0,False,False
4,"{'affiliations': [{'city': None, 'city_id': No...",2024-06-11,10.1542/peds.2023-065573,pub.1172744193,"{'id': 'jour.1017874', 'title': 'Pediatrics'}",NaN,"{'id': 'jour.1017874', 'title': 'Pediatrics'}",Performance of ChatGPT and Google Translate fo...,2024,NaN,...,United States,US,NaN,NaN,NaN,Pediatrics,3.0,9.0,False,False


In [ ]:
# Dictionary to map problematic country names to their ISO alpha-3 codes
country_name_corrections = {
    'Democratic Republic of the Congo': 'COD',
    'Turkey': 'TUR',
    'Russia': 'RUS',
    'Ivory Coast': 'CIV',
    'Palestinian Territory': 'PSE',
    'Reunion': 'REU',
    'Timor Leste': 'TLS',
    'Vatican': 'VAT',
    'Brunei': 'BRN',
    'Saint Helena': 'SHN',
    'Micronesia': 'FSM',
    'Sint Maarten': 'SXM'
}

# Function to convert country names to ISO alpha-3 country codes
def country_to_alpha_3(country_name):
    if country_name in country_name_corrections:
        return country_name_corrections[country_name]
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        if country_name is not np.nan:
            print(f'Error: {country_name} not found in pycountry')
        return np.nan

df_final['first_country_code'] = df_final['first_country'].apply(country_to_alpha_3)
df_final['second_country_code'] = df_final['second_country'].apply(country_to_alpha_3)
df_final['third_country_code'] = df_final['third_country'].apply(country_to_alpha_3)
df_final['funder_country_code'] = df_final['funder_country_code'].apply(country_to_alpha_3)

Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not found in pycountry
Error: None not foun

In [ ]:
df_final.to_csv('Preprocessed_papers_full.csv', index=False)

### Preprocessing (Extra)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import geopandas as gpd
import matplotlib.pyplot as plt
import pycountry
import pycountry_convert as pc

# create 'images/'
if not os.path.exists('images/'):
    os.makedirs('images/')

# Dictionary to map problematic country names to their ISO alpha-3 codes
country_name_corrections = {
    'Democratic Republic of the Congo': 'COD',
    'Turkey': 'TUR',
    'Russia': 'RUS',
    'Ivory Coast': 'CIV',
    'Palestinian Territory': 'PSE',
    'Reunion': 'REU',
    'Timor Leste': 'TLS',
    'Vatican': 'VAT',
    'Brunei': 'BRN',
    'Saint Helena': 'SHN',
    'Micronesia': 'FSM',
    'Sint Maarten': 'SXM'
}

# Function to convert country names to ISO alpha-3 country codes
def country_to_alpha_3(country_name):
    if country_name in country_name_corrections:
        return country_name_corrections[country_name]
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        if country_name is not np.nan:
            print(f'Error: {country_name} not found in pycountry')
        return np.nan

# Function to extract the first name from a full name
def extract_first_name(full_name):
    try:
        names = full_name.split()
        if names[0].endswith(','):
            if len(names) > 1 and not names[1].endswith('.'):
                return names[1]
            elif len(names) > 2:
                return names[2]
        elif names[0].endswith('.'):
            if len(names) > 1 and not names[1].endswith('.'):
                return names[1]
            elif len(names) > 2:
                return names[2]
        return names[0]
    except:
        return np.nan

# Function to convert alpha-3 to alpha-2 country code
def alpha3_to_alpha2(alpha_3):
    try:
        return pycountry.countries.get(alpha_3=alpha_3).alpha_2
    except:
        return None

# Function to get continent from alpha-3 country code
def get_continent(alpha_3):
    alpha_2 = alpha3_to_alpha2(alpha_3)
    if alpha_2:
        try:
            continent_code = pc.country_alpha2_to_continent_code(alpha_2)
            continent_name = pc.convert_continent_code_to_continent_name(continent_code)
            return continent_name
        except:
            return None
    return None

In [ ]:
df = pd.read_csv('Preprocessed_papers_full.csv')
# Get full name from 'first_name' and'last_name'
df['full_name'] = df['first_name'] + ' ' + df['last_name']

# Apply the extraction function
df['extracted_first_name'] = df['full_name'].apply(extract_first_name)
# remove the rows with missing values in the 'extracted_first_name' column
df = df.dropna(subset=['extracted_first_name'])

df.head()

In [ ]:
gender_ref = pd.read_csv('gender-api-output.csv', index_col=0)

# Function to convert 2-digit country codes to 3-digit country codes
def convert_alpha2_to_alpha3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return code

# Apply the conversion
gender_ref['country_code'] = gender_ref['country_code'].apply(convert_alpha2_to_alpha3)

In [ ]:
# Merge with gender_ref to get the gender
df = pd.merge(df, gender_ref, how='left', left_on=['extracted_first_name', 'first_country_code'], right_on=['name_standard', 'country_code'])
df.rename(columns={'ga_gender': 'gender'}, inplace=True)

df

In [ ]:
## Create sub-groups
wb_df = pd.read_csv('List of economies.csv', sep=';').drop(columns='Unnamed: 5')

# Classify countries into HICs and LMICs based on the identified groups
hic_country_codes = set(wb_df[wb_df['Income group'] == 'High income']['Code'])
lmic_country_codes = set(wb_df[wb_df['Income group'].isin(['Low income', 'Lower middle income', 'Upper middle income'])]['Code'])

# Check if any of the country codes in the row are in the given set
def is_in_country_list(row, country_set):
    return (
        row['first_country_code'] in country_set or
        row['second_country_code'] in country_set or
        row['third_country_code'] in country_set
    )

# create a new column in the DataFrame to classify the countries LMIC true or false. If any of the countries in the row is a LMIC, the row is classified as LMIC
df['LMIC'] = df.apply(is_in_country_list, country_set=lmic_country_codes, axis=1)
# create a new column in the DataFrame to classify the countries HIC true or false. If any of the countries in the row is a HIC, the row is classified as HIC
df['HIC'] = df.apply(is_in_country_list, country_set=hic_country_codes, axis=1)

In [ ]:
# Adding continent columns
continents = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']
for continent in continents:
    df[continent] = 0
    df[f'{continent}_LMIC'] = 0
    df[f'{continent}_HIC'] = 0

# Populate the continent columns
for continent in continents:
    df[continent] = df['first_country_code'].apply(lambda x: get_continent(x) == continent if x is not np.nan else False).astype(int)
    df[continent] = df[continent] | df['second_country_code'].apply(lambda x: get_continent(x) == continent if x is not np.nan else False).astype(int)
    df[continent] = df[continent] | df['third_country_code'].apply(lambda x: get_continent(x) == continent if x is not np.nan else False).astype(int)

    df[f'{continent}_LMIC'] = df.apply(
        lambda row: (get_continent(row['first_country_code']) == continent and row['first_country_code'] in lmic_country_codes) or
                    (get_continent(row['second_country_code']) == continent and row['second_country_code'] in lmic_country_codes) or
                    (get_continent(row['third_country_code']) == continent and row['third_country_code'] in lmic_country_codes), axis=1).astype(int)

    df[f'{continent}_HIC'] = df.apply(
        lambda row: (get_continent(row['first_country_code']) == continent and row['first_country_code'] in hic_country_codes) or
                    (get_continent(row['second_country_code']) == continent and row['second_country_code'] in hic_country_codes) or
                    (get_continent(row['third_country_code']) == continent and row['third_country_code'] in hic_country_codes), axis=1).astype(int)

In [ ]:
continents = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']
income_groups = ['HIC', 'LMIC']

def get_continent_and_income(country_code):
    continent = get_continent(country_code)
    if country_code in lmic_country_codes:
        return f"{continent}_LMIC"
    elif country_code in hic_country_codes:
        return f"{continent}_HIC"
    return None

for continent in continents:
    for income_group in income_groups:
        df[f'{continent}_{income_group}_funder'] = 0

# Populate the funder continent and income columns
for i, row in df.iterrows():
    if pd.notna(row['funder_country_code']):
        continent_income = get_continent_and_income(row['funder_country_code'])
        if continent_income:
            df.at[i, f'{continent_income}_funder'] = 1